# Concentrated liquidity: Fee earned versus IL

Liquidity rebalancing
https://atise.medium.com/liquidity-provider-strategies-for-uniswap-liquidity-rebalancing-f4430eec63a0

Liquidity rebalancing or relocation refers to moving the liquidity around to a different price boundaries. It’s typically used to move out-of-range LP positions back into the range, where the active price is.



In [6]:
!pip install giza-datasets


In [7]:
import yfinance as yf
import math
import numpy as np
import pandas as pd
from giza.datasets import DatasetsLoader
from giza.datasets import DatasetsHub
hub = DatasetsHub()

hub.show()

                           Datasets                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Dataset Name                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ tvl-fee-per-protocol                                       │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens                                     │
├────────────────────────────────────────────────────────────┤
│ tokens-ohcl                                                │
├────────────────────────────────────────────────────────────┤
│ top-pools-apy-per-protocol                                 │
├────────────────────────────────────────────────────────────┤
│ tokens-daily-prices-mcap-volume                            │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=aave-v2                     │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=aave-v3                     │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=aura                        │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=balancer-v2                 │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=beefy                       │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=benqi-lending               │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=coinwind                    │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=compound-v2                 │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=convex-finance              │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=curve-dex                   │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=flamincome                  │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=lido                        │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=pancakeswap-amm             │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=pendle                      │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=rocket-pool                 │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=sushiswap                   │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=uniswap-v2                  │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=uniswap-v3                  │
├────────────────────────────────────────────────────────────┤
│ tvl-per-project-tokens/project=yearn-finance               │
├────────────────────────────────────────────────────────────┤
│ aave-daily-deposits-borrowsv2                              │
├────────────────────────────────────────────────────────────┤
│ aave-daily-deposits-borrowsv3                              │
├────────────────────────────────────────────────────────────┤
│ aave-daily-rates-indexes                                   │
├────────────────────────────────────────────────────────────┤
│ aave-liquidationsV2                                        │
├────────────────────────────────────────────────────────────┤
│ aave-liquidationsV3                                        │
├────────────────────────────────────────────────────────────┤
│ balancer-daily-pool-liquidity                              │
├──────────────────────────────

In [8]:
# Define the time period for historical data
start_date = '2024-02-22'
end_date = '2024-05-22'

# Fetch historical data for ETH and STRK
eth_data = yf.download('ETH-USD', start=start_date, end=end_date, interval='1d')
strk_data = yf.download('STRK22691-USD', start=start_date, end=end_date, interval='1d')

# Display the first few rows of each dataset
print("ETH Data:")
print(eth_data.head())
print("\nSTRK Data:")
print(strk_data.head())

# Usage example:
loader = DatasetsLoader()

liquidity_df = loader.load('uniswapv3-ethereum-wbtc-wsteth-500-liquidity-snapshots')
ticks_df = loader.load('uniswapv3-ethereum-wbtc-wsteth-500-current-ticks')
pools_info_df = loader.load('uniswapv3-ethereum-pools-info')

liquidity_df.head()
ticks_df.head()
pools_info_df.head()

# Display the first few rows of each DataFrame
print("Liquidity Snapshots:")
print(liquidity_df.head())

print("\nCurrent Ticks:")
print(ticks_df.head())

print("\nPools Info:")
print(pools_info_df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ETH Data:
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2024-02-22  2969.599854  3030.666016  2907.109375  2971.007324  2971.007324   
2024-02-23  2970.139648  2991.329590  2906.583740  2921.658203  2921.658203   
2024-02-24  2921.962891  3003.195068  2907.700684  2992.385986  2992.385986   
2024-02-25  2992.366699  3117.428955  2984.393066  3112.697266  3112.697266   
2024-02-26  3112.529053  3197.375000  3037.954590  3178.993652  3178.993652   

                 Volume  
Date                     
2024-02-22  18058908246  
2024-02-23  12822717059  
2024-02-24  10701688842  
2024-02-25  14620450464  
2024-02-26  17504464351  

STRK Data:
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2024-02-22  1.897438  2.014914  1.805397  1.920013   1.920013  652017969
2024-02-23  1.922879  2

Protocol: MySwap
Pool: ETH-STRK

Assuming u would contribute liquidity to a price range of +-5% of current price, What would be my returns in a time period of 3 months from fees.

Risk: Impermanent loss (IL). Within a given price range of ur liquidity, say current price is 3000 and u invest between 2850 and 3150 in the pool. As price moves from 3000 to up or down, u will get something called IL.

But as swaps happen, u also earn fees as long as price is within this range. When price moves out of this range, ur simulation shall re-adjust position to new current price. e.g. if price moves to 2850, u rebalance here to be +-5% based on this price. The amount used to rebalance will be ur total available amount during the rebalance time. This is dependent on how much fees u earn vs how much u lost in IL.

There are more minute details, i want u to digest this so far and have an high level approach to solve this

In [9]:
# Assuming eth_data and strk_data are already fetched

# Parameters
price_range = 0.05  # ±5%
initial_liquidity = 10  # Initial liquidity provided in ETH
trading_volume_per_day = 1000000  # Average daily trading volume in the pool
fee_percentage = 0.003  # 0.3% fee
time_period_days = len(eth_data)  # Use the length of the fetched data
print(time_period_days)

# Function to calculate impermanent loss 
# derivation: https://medium.com/auditless/how-to-calculate-impermanent-loss-full-derivation-803e8b2497b7
def impermanent_loss(price_ratio):
    return 2 * (np.sqrt(price_ratio) / (1 + price_ratio)) - 1
def impermanent_loss(r):
    return 1 - math.sqrt((4 * r) / ((r + 1) ** 2))

# Initialize variables
fees_earned = 0
impermanent_loss_total = 0
current_liquidity = initial_liquidity
initial_price = eth_data['Close'].iloc[0]

# Simulate the process
for day in range(time_period_days):
    current_price = eth_data['Close'].iloc[day]
    
    # Calculate fees earned for the day
    daily_fees = trading_volume_per_day * fee_percentage
    fees_earned += daily_fees
    
    # Check if price is within the range
    lower_bound = initial_price * (1 - price_range)
    upper_bound = initial_price * (1 + price_range)
    
    # Rebalancing code: Check how exactly to do it 
    if not (lower_bound <= current_price <= upper_bound):
        # Price out of range, rebalance
        price_ratio = current_price / initial_price
        il = impermanent_loss(price_ratio)
        impermanent_loss_total += il
        
        # Update liquidity position
        current_liquidity += fees_earned  # Add earned fees to liquidity
        fees_earned = 0  # Reset daily fees
        initial_price = current_price  # Update price for new range

# Final calculation of returns
net_earnings = current_liquidity + fees_earned - initial_liquidity - impermanent_loss_total

# Display results
results = {
    "Initial Liquidity (ETH)": initial_liquidity,
    "Final Liquidity (ETH)": current_liquidity,
    "Total Fees Earned (ETH)": fees_earned,
    "Impermanent Loss (ETH)": impermanent_loss_total,
    "Net Earnings (ETH)": net_earnings
}

results_df = pd.DataFrame(results, index=[0])
print(results_df)


90
   Initial Liquidity (ETH)  Final Liquidity (ETH)  Total Fees Earned (ETH)  \
0                       10               267010.0                   3000.0   

   Impermanent Loss (ETH)  Net Earnings (ETH)  
0                 0.01734        269999.98266  


In [10]:
import numpy as np
import math

def impermanent_loss(r):
    return 1 - math.sqrt((4 * r) / ((r + 1) ** 2))

def fees_earned(initial_value, fee_percentage, volume):
    return initial_value * fee_percentage * volume

def simulate_liquidity_provision(initial_price, lower_bound, upper_bound, fee_percentage, time_period_months, price_changes, daily_volume_multiple):
    current_price = initial_price
    total_fees = 0
    total_value = initial_value = 1000  # Initial liquidity value in USD

    for day in range(time_period_months * 30):
        # Simulate price movement for the day
        current_price *= price_changes[day]

        if lower_bound <= current_price <= upper_bound:
            # Within range: earn fees
            daily_volume = initial_value * daily_volume_multiple
            daily_fees = fees_earned(initial_value, fee_percentage, daily_volume)
            total_fees += daily_fees
            total_value += daily_fees
        else:
            # Out of range: rebalance
            price_ratio = current_price / initial_price
            il = impermanent_loss(price_ratio)
            total_value = total_value * (1 - il)
            total_value += total_fees  # Reinvest fees earned so far
            total_fees = 0  # Reset fees after rebalancing

            # Rebalance to new range
            lower_bound = current_price * 0.95
            upper_bound = current_price * 1.05
            initial_price = current_price

    return total_value

# Parameters
initial_price = 3000
lower_bound = initial_price * 0.95
upper_bound = initial_price * 1.05
fee_percentage = 0.003  # 0.3% fee
time_period_months = 3
daily_volume_multiple = 0.01  # Assume 1% of initial value traded daily

# Simulate daily price changes (example: small daily fluctuations)
np.random.seed(42)  # For reproducibility
price_changes = 1 + np.random.normal(0, 0.01, time_period_months * 30)  # 1% daily volatility

# Run simulation
final_value = simulate_liquidity_provision(initial_price, lower_bound, upper_bound, fee_percentage, time_period_months, price_changes, daily_volume_multiple)
print(f"Final Value after {time_period_months} months: ${final_value:.2f}")


Final Value after 3 months: $4958.26


In [11]:

# Uniswap math functions
# https://youtu.be/_asFkMz4zhw?t=25 - Uniswap V3 - Liquidity | DeFi

def get_liquidity_0(x, sp, sb):
    return x * sp * sb / (sb - sp)

def get_liquidity_1(y, sp, sa):
    return y / (sp - sa)

def get_liquidity(x, y, sp, sa, sb):
    if sp <= sa:
        liquidity = get_liquidity_0(x, sp, sb)
    elif sp < sb:
        liquidity0 = get_liquidity_0(x, sp, sb)
        liquidity1 = get_liquidity_1(y, sp, sa)
        liquidity = min(liquidity0, liquidity1)
    else:
        liquidity = get_liquidity_1(y, sp, sa)
    return liquidity


#
# Calculate x and y given liquidity and price range
#
def calculate_x(L, sp, sa, sb):
    sp = max(min(sp, sb), sa)     # if the price is outside the range, use the range endpoints instead
    return L * (sb - sp) / (sp * sb)

def calculate_y(L, sp, sa, sb):
    sp = max(min(sp, sb), sa)     # if the price is outside the range, use the range endpoints instead
    return L * (sp - sa)

def calculate_a1(L, sp, sb, x, y):
    # https://www.wolframalpha.com/input/?i=solve+L+%3D+y+%2F+%28sqrt%28P%29+-+a%29+for+a
    # sqrt(a) = sqrt(P) - y / L
    return (sp - y / L) ** 2

def calculate_a2(sp, sb, x, y):
    # https://www.wolframalpha.com/input/?i=solve+++x+sqrt%28P%29+sqrt%28b%29+%2F+%28sqrt%28b%29++-+sqrt%28P%29%29+%3D+y+%2F+%28sqrt%28P%29+-+a%29%2C+for+a
    # sqrt(a) = (y/sqrt(b) + sqrt(P) x - y/sqrt(P))/x
    #    simplify:
    # sqrt(a) = y/(sqrt(b) x) + sqrt(P) - y/(sqrt(P) x)
    sa = y / (sb * x) + sp - y / (sp * x)
    return sa ** 2

#
# Two different ways how to calculate p_b. calculate_b1() uses liquidity as an input, calculate_b2() does not.
#
def calculate_b1(L, sp, sa, x, y):
    # https://www.wolframalpha.com/input/?i=solve+L+%3D+x+sqrt%28P%29+sqrt%28b%29+%2F+%28sqrt%28b%29+-+sqrt%28P%29%29+for+b
    # sqrt(b) = (L sqrt(P)) / (L - sqrt(P) x)
    return ((L * sp) / (L - sp * x)) ** 2

def calculate_b2(sp, sa, x, y):
    # find the square root of b:
    # https://www.wolframalpha.com/input/?i=solve+++x+sqrt%28P%29+b+%2F+%28b++-+sqrt%28P%29%29+%3D+y+%2F+%28sqrt%28P%29+-+sqrt%28a%29%29%2C+for+b
    # sqrt(b) = (sqrt(P) y)/(sqrt(a) sqrt(P) x - P x + y)
    P = sp ** 2
    return (sp * y / ((sa * sp - P) * x + y)) ** 2



def calculate_P(x,y,sa,sb):
  p = sb*x
  q = y-x*sa*sb
  r = -sb*y
  return (math.pow(-1*q+(q**2 - 4*p*r),0.5)/(2*p) )

def tick_price(t):
  return 1.0001**t

def price_tick(p):
  return math.log(p,1.0001)

def getYFromX(x,sa,sb,sp):
    return x * (sp - sa) * sp *sb /(sb - sp)

# ? P is Y/X
def simulator_calc_L(x,pa,pb,p):
    sa = pa ** 0.5
    sb = pb ** 0.5
    sp = p ** 0.5
    if p >= pb:
        y=x
        x=0
        L = get_liquidity(x, y, sp, sa, sb)
    elif p <= pa:
        y = 0
    else:
        print("sim2", sa, sb, sp, x)
        y = getYFromX(x, sa, sb, sp)
        L = get_liquidity(x, y, sp, sa, sb)

    return L,y

# ! Requires X token to be a stable token
def getXY(capitalDollar, token0Decimals, token1Decimals, Pxusd, Pyusd, sp, sa, sb):
    # captal = x * Pxusd / xdecimals + y * Pyusd / yDecimals  ----- eq1
    # replace y with x from getYFromX

    factor1 = Pxusd / (10 ** token0Decimals)
    factor2 = Pyusd / (10 ** token1Decimals)
    factor3 = (sp - sa) * sp * sb / (sb - sp)

    x = capitalDollar / (factor1 + factor3 * factor2)
    y = getYFromX(x, sa, sb, sp)
    return x, y